# T5 Tuned - Inference

## Install Dependencies

In [1]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.0 MB/s eta 0:00:00


## Connect to Google Drive
We will be loading data from google drive and also save trained models to google drive. So lets mount google drive.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports and Constants

In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import transformers
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from collections import deque

DATA_NAME = "s2"
# T5_MODEL_NAME = "t5-small"
T5_MODEL_NAME = "t5-base"

TUNED_T5_SAVED = f'drive/MyDrive/MIDS/w266/project/saved_models/{T5_MODEL_NAME}-data{DATA_NAME}-finetuned'
PROMPT = 'Generate next line: '

FINAL_TEST_LIST = ['Princess Leia lay upon her bed all the night.',
                   'He stopped himself for a minute and thought if it was the right thing to do.',
                   'There once lived king named Rama.',
                   'Once upon a time, an old Owl lived in the forest.']

In [4]:
t5_tokenizer = T5Tokenizer.from_pretrained(T5_MODEL_NAME)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


# Inference

In [5]:
class Inferencer:
  def __init__(self, model, tokenizer, prompt='', max_new_tokens=100, tensor_type='pt', num_beams=3):
    self.model = model
    self.tokenizer = tokenizer
    self.prompt = prompt
    self.max_new_tokens = max_new_tokens
    self.tensor_type = tensor_type
    self.num_beams = num_beams

  def __call__(self, context_lines):
    transformers.logging.set_verbosity_error()
    test_inputs = self.tokenizer([self.prompt + ' '.join(context_lines)], return_tensors=self.tensor_type)
    test_output_ids = model.generate(
        test_inputs['input_ids'].cuda(),
        num_beams=self.num_beams,
        no_repeat_ngram_size=2,
        num_return_sequences=self.num_beams,
        max_new_tokens=self.max_new_tokens,
        do_sample=True,
        top_k=0)
    decoded = [self.tokenizer.decode(out_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) for out_ids in test_output_ids]
    return decoded


In [29]:
class StoryBot:
  """Class to mimic a bot that continues the story."""
  def __init__(self, inferencer, n_iters=20, lines_to_use=1):
    self._n_iters = n_iters
    self.lines_to_use = lines_to_use
    self.inferencer = inferencer
    self.re_init()

  def re_init(self):
    self.story = []
    self.context_lines = deque([], self.lines_to_use)

  def get_user_choice(self):
    output_lines = self.inferencer(self.context_lines)
    if len(output_lines) > 1:
      print('Choose the line of your choice:')
      for i, line in enumerate(output_lines):
        print(f'{i}:', line)
      print(f'{i+1}: Regenerate')
      print(f'{i+2}: End')
      user_opt = -1
      while user_opt == -1:
        try:
            user_input = input('Input the number of your choice (or ): ')
            user_opt = int(user_input)
            if user_opt < len(output_lines):
              return output_lines[user_opt]
            elif user_opt == len(output_lines):
              return 'regenerate'
            elif user_opt == len(output_lines) + 1:
              return 'end'
        except ValueError:
            user_opt = -1
    else:
      return output_lines[0]

  def print_story(self):
    for i, line in enumerate(self.story):
      if i%2 == 0:
        print(f'User: {line}') 
      else:
        print(f'Generated: {line}') 

  def run(self):
    print('*'*50)
    print('Welcome to StoryBot!\n')
    print('This program simulates an MMS kind of interaction with a bot to create a story sequentially.')
    print('When the prompt appears below, start typing as if it were the input on your mobile.')
    print('Enter end to end the story and restart to restart.') 
    print('*'*50, '\n')
    restart = False
    i = 0
    while i < self._n_iters:
      if i > 0:
        print('The story so far:')
        self.print_story()
      i = i + 1
      # get the sentence from the user
      sentence_in = input('Enter next line (or end): ').strip()
      # accomodate special prompts
      if sentence_in == 'end':
        break
      if sentence_in == 'restart':
        i = 0
        self.re_init()
        continue
      self.context_lines.append(sentence_in)
      self.story.append(sentence_in)
      output = 'regenerate'
      while output == 'regenerate':
        output = self.get_user_choice()
      if output == 'end':
        break
      self.context_lines.append(output)
      self.story.append(output)

    print()
    print('\n======== Final story: =========\n')
    self.print_story()



In [ ]:
tokenizer = T5Tokenizer.from_pretrained(T5_MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(TUNED_T5_SAVED).cuda()


In [30]:
inferencer = Inferencer(model, tokenizer, prompt=PROMPT)
story_bot = StoryBot(inferencer, n_iters=10, lines_to_use=5)
story_bot.run()

**************************************************
Welcome to StoryBot!

This program simulates an MMS kind of interaction with a bot to create a story sequentially.
When the prompt appears below, start typing as if it were the input on your mobile.
Enter end to end the story and restart to restart.
************************************************** 

Enter next line (or end): There was a farmer who had a dog.
Choose the line of your choice:
0: He had a dog, and he was very polite.
1: He was a dog, and he was very fond of dogs.
2: He had a dog, and he was very proud of it.
3: Regenerate
4: End
Input the number of your choice (or ): 1
The story so far:
User: There was a farmer who had a dog.
Generated: He was a dog, and he was very fond of dogs.
Enter next line (or end): He had multiple dogs of the same breed
Choose the line of your choice:
0: He had a dog, and he was very fond of dogs.
1: He had a lot of dogs, and he liked to have them.
2: He had a dog, and he was very fond of dogs.
3:

In [25]:
def evaluate(model, tokenizer, lines, prompt, contrastive=True):
  transformers.logging.set_verbosity_error()
  for test_input_text in lines:
      test_inputs = tokenizer([prompt + test_input_text], return_tensors='pt')
      if contrastive:
        test_output_ids = model.generate(
            test_inputs['input_ids'].cuda(), 
            penalty_alpha=0.4, top_k=5, max_length=256)
      else:
        test_output_ids = model.generate(
            test_inputs['input_ids'].cuda(),
            num_beams=5,
            no_repeat_ngram_size=2,
            num_return_sequences=5,
            max_new_tokens=100,
            do_sample=True,
            top_k=0)
      print(f'Input: {test_input_text}')
      decoded = [tokenizer.decode(out_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) for out_ids in test_output_ids]
      print(f'Output: {decoded}')

In [ ]:
def infer_nextline(model, tokenizer, lines, prompt, contrastive=True):
  transformers.logging.set_verbosity_error()
  for test_input_text in lines:
      test_inputs = tokenizer([prompt + test_input_text], return_tensors='pt')
      if contrastive:
        test_output_ids = model.generate(
            test_inputs['input_ids'].cuda(), 
            penalty_alpha=0.4, top_k=5, max_length=256)
      else:
        test_output_ids = model.generate(
            test_inputs['input_ids'].cuda(),
            num_beams=5,
            no_repeat_ngram_size=2,
            num_return_sequences=5,
            max_new_tokens=100,
            do_sample=True,
            top_k=0)
      print(f'Input: {test_input_text}')
      decoded = [tokenizer.decode(out_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) for out_ids in test_output_ids]
      print(f'Output: {decoded}')

**************************************************
Welcome to StoryBot!

This program simulates an MMS kind of interaction with a bot to create a story sequentially.
When the prompt appears below, start typing as if it were the input on your mobile.

See the following special prompts:
1) opts: Model will give you 3 options to choose from (takes longer to generate)
2) gocrazy: Running out of ideas? Give the story an unexpected twist (not always make sense)
3) readstory: Print the full story
4) endstory: Print the story and finish the program
************************************************** 

Once upon a time, an old Owl lived in the forest.


IndexError: ignored

In [ ]:
## Fine tuned T5 model
print("==== Contrastive Search ====")
evaluate(model, t5_tokenizer, FINAL_TEST_LIST, PROMPT, contrastive=True)

print("==== Beam Search ====")
evaluate(model, t5_tokenizer, FINAL_TEST_LIST, PROMPT, contrastive=False)

==== Contrastive Search ====
Input: Princess Leia lay upon her bed all the night.
Output: ['She slept in a slumbering, slumbering, slumbering, slumbering, slumbering, slumbering, slumbering, slumbering, slumbering, slumbering, slumbering, slumbering, slumbering, slumbering, slumbering, slumbering, slumbering, slumbering']
Input: He stopped himself for a minute and thought if it was the right thing to do.
Output: ['“It is a good thing to do,” said he, “and I am sure you will be able to do it.”']
Input: There once lived king named Rama.
Output: ['He was a great king, and he was the most powerful.']
Input: Once upon a time, an old Owl lived in the forest.
Output: ['He sat on a tree, and he sat down to eat the berries.']
==== Beam Search ====
Input: Princess Leia lay upon her bed all the night.
Output: ['She was so tired that she could not sleep.', 'She was so dreadful that she felt as if she had lost her way, and she did not know what to do.', 'Then she went to sleep, and when she had wok

In [26]:
evaluate(model, t5_tokenizer, ['A rich old farmer, who felt that he had not many more days to live, called his sons to his bedside.'], PROMPT, contrastive=False)

Input: A rich old farmer, who felt that he had not many more days to live, called his sons to his bedside.
Output: ['They sat down on a bed, and looked at each other.', 'They said to him, “It is a good thing that thou shalt not die.”', '"I am a farmer," he said, "and I do not know what to do with my sons."', 'The sons of the poor farmer were very happy, but they did not want to see him, and so he gave them a supper.', 'The sons sat in the bed, and cried out: "Let us go to bed."']


In [ ]:
evaluate(model, t5_tokenizer, ['Then he sat down and talked to his sons.'], PROMPT, contrastive=False)

Input: Then he sat down and talked to his sons.
Output: ['"I don\'t know," said he.', 'Then he sat down and talked to them.', 'Then he said to his sons, “I am very sorry for you, but I don’t know what to do with you.”', '"It\'s a good thing," he said.', '"What do you think?" asked he.']


In [ ]:
story_bot = GPTStoryBot()
story_bot.run()

## Round 1

**Prompt**: Once upon a time, an old Owl lived in the forest.

**Response**: He was a good-natured creature.

**Prompt**: The owl helped all animals that came to him.

**Response**: The owl was a very good man, and he was very kind to the animals that came to him.

**Prompt**: One day a little mouse came to the owl.

**Response**: The little mouse sat down and looked at the owl.

**Prompt**: The owl saw that the mouse was very sad.

**Response**: The owl said, “It is a very sad mouse."

**Prompt**: The owl asked "why are you so sad, mouse?"

**Response**: "It\'s a misfortune," answered the mouse.

## Round 2
**Prompt**: A rich old farmer, who felt that he had not many more days to live, called his sons to his bedside.
**Response**: 